# Financial Ratio Quantile Trading Strategy Analysis

**Course:** Quantitative Trading Strategies  
**Assignment:** Week 3 - Financial Ratio Quantiles  
**Period:** January 2018 - June 2023  
**Universe:** ~1200 US Equities  

---

## Executive Summary

This notebook implements a quantamental trading strategy based on financial accounting ratios:
- **Debt-to-Market-Cap**: Leverage indicator
- **Return on Investment (ROI)**: Operating efficiency
- **Price-to-Earnings (P/E)**: Valuation metric

The strategy constructs long-short portfolios by ranking stocks on these fundamental signals, going long the most attractive decile and short the least attractive decile.

**Key Implementation Features:**
- Filing-date-aware ratio computation (no look-ahead bias)
- Daily market cap adjustments between filings
- Multiple signal combination methods (weighted avg, PCA, rank-based)
- Position sizing variations (vigintile doubling/halving)
- Ratio changes vs absolute values analysis

---

## 1. Setup & Imports

In [40]:
# Standard library imports
import warnings
from pathlib import Path
from dataclasses import dataclass, field
from enum import Enum
from typing import Optional, List, Dict, Tuple
import json
import pickle

# Scientific computing
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 4)

print("✓ Imports successful")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

✓ Imports successful
NumPy version: 2.3.5
Pandas version: 2.3.3


### Plotting Style Setup

In [41]:
def setup_plot_style():
    """Set up consistent plot styling."""
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.rcParams['figure.figsize'] = (14, 6)
    plt.rcParams['font.size'] = 10
    plt.rcParams['axes.titlesize'] = 12
    plt.rcParams['axes.labelsize'] = 10
    plt.rcParams['lines.linewidth'] = 1.5
    sns.set_palette("husl")

setup_plot_style()
print("✓ Plot style configured")

✓ Plot style configured


## 2. Configuration Parameters

In [42]:
# File paths
DATA_DIR = Path('data')
OUTPUT_DIR = Path('outputs')
PLOTS_DIR = OUTPUT_DIR / 'plots'
RESULTS_DIR = OUTPUT_DIR / 'results'

# Create directories
PLOTS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Date range (assignment specification)
START_DATE = '2018-01-01'
END_DATE = '2023-06-30'

# Universe filters (Section 3 of assignment)
MIN_MARKET_CAP_MM = 100  # $100MM minimum
MIN_DEBT_RATIO = 0.1     # Must exceed 0.1 somewhere in period
MIN_DEBT_RATIO_COUNT = 3 # "More than fleetingly" = at least 3 quarters

# Excluded sectors (assignment Section 3)
EXCLUDED_SECTORS = ['Automotive', 'Finance', 'Insurance']

# Strategy parameters
REBALANCE_FREQ = 'M'  # 'W' for weekly, 'M' for monthly
LONG_QUANTILE = 0.90  # Top decile (90th percentile and above)
SHORT_QUANTILE = 0.10 # Bottom decile (10th percentile and below)

# Capital management (assignment Section 5)
LEVERAGE_MULTIPLE = 10  # Initial capital = 10x gross notional
FUNDING_RATE = 0.02     # 2% annual (constant) or use rolling LIBOR/SOFR
REPO_SPREAD = 0.01      # Repo rate = funding rate - 100bp

# Display configuration
print("Configuration:")
print(f"  Data directory: {DATA_DIR}")
print(f"  Analysis period: {START_DATE} to {END_DATE}")
print(f"  Rebalancing: {REBALANCE_FREQ}")
print(f"  Long/Short quantiles: {LONG_QUANTILE}/{SHORT_QUANTILE}")
print(f"  Min market cap: ${MIN_MARKET_CAP_MM}MM")
print(f"  Excluded sectors: {EXCLUDED_SECTORS}")

Configuration:
  Data directory: data
  Analysis period: 2018-01-01 to 2023-06-30
  Rebalancing: M
  Long/Short quantiles: 0.9/0.1
  Min market cap: $100MM
  Excluded sectors: ['Automotive', 'Finance', 'Insurance']


## 3. Core Data Classes (Adapted from Week2)

These classes are embedded from `week2/src/crypto_spread/strategy.py` and adapted for equity long-short.

In [43]:
class PositionSide(Enum):
    """Position side enumeration."""
    FLAT = "FLAT"
    LONG = "LONG"
    SHORT = "SHORT"


class ExitReason(Enum):
    """Exit reason enumeration."""
    NONE = "NONE"
    REBALANCE = "REBALANCE"  # Monthly/weekly rebalancing
    END_OF_DATA = "END_OF_DATA"


@dataclass
class Position:
    """Represents a trading position in a single stock."""
    ticker: str
    side: PositionSide
    entry_price: float
    entry_time: pd.Timestamp
    shares: float  # Number of shares (can be fractional)
    
    def __post_init__(self):
        if self.side == PositionSide.FLAT:
            raise ValueError("Cannot create a FLAT position object")


@dataclass
class Trade:
    """Represents a completed trade."""
    ticker: str
    side: PositionSide
    entry_time: pd.Timestamp
    exit_time: pd.Timestamp
    entry_price: float
    exit_price: float
    shares: float
    pnl: float
    exit_reason: ExitReason


@dataclass
class BacktestResult:
    """Container for backtest results."""
    trades: List[Trade]
    equity_curve: pd.Series
    final_capital: float
    total_return: float
    sharpe_ratio: float
    max_drawdown: float
    win_rate: float
    num_trades: int
    num_rebalances: int
    params: Dict = field(default_factory=dict)


print("✓ Core data classes defined")

✓ Core data classes defined


## 4. Performance Metrics Functions (From Week2)

Embedded from `week2/src/crypto_spread/metrics.py` with adaptations for daily equity returns.

In [44]:
def calculate_sharpe_ratio(
    equity_curve: pd.Series,
    risk_free_rate: float = 0.0,
    periods_per_year: int = 252,
) -> float:
    """
    Calculate annualized Sharpe ratio using daily returns.
    
    Formula: Sharpe = sqrt(252) * mean(excess_returns) / std(excess_returns)
    
    Args:
        equity_curve: Series of equity values with datetime index
        risk_free_rate: Annual risk-free rate (default: 0%)
        periods_per_year: Trading days per year (252)
    
    Returns:
        Annualized Sharpe ratio
    """
    if equity_curve.empty or len(equity_curve) < 2:
        return 0.0
    
    # Calculate daily returns
    daily_returns = equity_curve.pct_change().dropna()
    
    if len(daily_returns) < 1:
        return 0.0
    
    # Excess returns
    rf_daily = risk_free_rate / periods_per_year
    excess_returns = daily_returns - rf_daily
    
    mean_excess = excess_returns.mean()
    std_excess = excess_returns.std()
    
    if std_excess == 0 or np.isnan(std_excess):
        return 0.0
    
    sharpe = np.sqrt(periods_per_year) * mean_excess / std_excess
    return float(sharpe)


def calculate_max_drawdown(equity_curve: pd.Series) -> float:
    """
    Calculate maximum drawdown as a percentage.
    
    Args:
        equity_curve: Series of equity values over time
    
    Returns:
        Maximum drawdown as a decimal (e.g., 0.10 for 10% drawdown)
    """
    if equity_curve.empty or len(equity_curve) < 2:
        return 0.0
    
    running_max = equity_curve.cummax()
    drawdown = (equity_curve - running_max) / running_max
    max_dd = drawdown.min()
    
    return abs(float(max_dd))


def calculate_win_rate(trades: List[Trade]) -> float:
    """
    Calculate win rate (percentage of profitable trades).
    
    Args:
        trades: List of completed trades
    
    Returns:
        Win rate as a decimal (e.g., 0.60 for 60%)
    """
    if not trades:
        return 0.0
    
    profitable = sum(1 for t in trades if t.pnl > 0)
    return profitable / len(trades)


def calculate_calmar_ratio(total_return: float, max_drawdown: float) -> float:
    """
    Calculate Calmar ratio (annualized return / max drawdown).
    
    Args:
        total_return: Total return as decimal
        max_drawdown: Maximum drawdown as decimal
    
    Returns:
        Calmar ratio
    """
    if max_drawdown == 0:
        return float('inf') if total_return > 0 else 0.0
    
    return total_return / max_drawdown


def calculate_downside_beta(
    portfolio_returns: pd.Series,
    market_returns: pd.Series,
) -> float:
    """
    Calculate downside beta (sensitivity during negative market returns).
    
    Args:
        portfolio_returns: Strategy returns (aligned with market)
        market_returns: Market benchmark returns (e.g., S&P 500)
    
    Returns:
        Downside beta
    """
    # Align series
    aligned = pd.DataFrame({
        'portfolio': portfolio_returns,
        'market': market_returns
    }).dropna()
    
    if len(aligned) < 10:
        return 0.0
    
    # Filter to negative market days
    negative_days = aligned[aligned['market'] < 0]
    
    if len(negative_days) < 5:
        return 0.0
    
    # Compute covariance and variance
    cov = negative_days['portfolio'].cov(negative_days['market'])
    var = negative_days['market'].var()
    
    if var == 0:
        return 0.0
    
    return cov / var


def calculate_tail_risk(returns: pd.Series, percentile: float = 0.01) -> float:
    """
    Calculate tail risk (Value at Risk at given percentile).
    
    Args:
        returns: Return series
        percentile: Percentile for VaR (0.01 for 1%, 0.05 for 5%)
    
    Returns:
        VaR (negative value representing loss)
    """
    if len(returns) < 10:
        return 0.0
    
    return returns.quantile(percentile)


print("✓ Performance metrics functions defined")

✓ Performance metrics functions defined


## 5. Data Loading Functions

In [45]:
def load_zacks_data() -> Dict[str, pd.DataFrame]:
    """
    Load all ZACKS fundamental data files.
    
    Returns:
        Dictionary mapping table name to DataFrame
    """
    print("Loading ZACKS data files...")
    
    data = {}
    
    # Financial Condition (balance sheet, income statement, cash flow)
    fc_file = list(DATA_DIR.glob('ZACKS_FC_2_*.csv'))[0]
    print(f"  Loading {fc_file.name}...")
    data['fc'] = pd.read_csv(fc_file, parse_dates=['per_end_date', 'filing_date'])
    print(f"    → {len(data['fc']):,} rows")
    
    # Financial Ratios (pre-computed)
    fr_file = list(DATA_DIR.glob('ZACKS_FR_2_*.csv'))[0]
    print(f"  Loading {fr_file.name}...")
    data['fr'] = pd.read_csv(fr_file, parse_dates=['per_end_date'])  # FR has no filing_date
    print(f"    → {len(data['fr']):,} rows")
    
    # Market Value snapshots
    mktv_file = list(DATA_DIR.glob('ZACKS_MKTV_2_*.csv'))[0]
    print(f"  Loading {mktv_file.name}...")
    data['mktv'] = pd.read_csv(mktv_file, parse_dates=['per_end_date'])
    print(f"    → {len(data['mktv']):,} rows")
    
    # Shares outstanding
    shrs_file = list(DATA_DIR.glob('ZACKS_SHRS_2_*.csv'))[0]
    print(f"  Loading {shrs_file.name}...")
    data['shrs'] = pd.read_csv(shrs_file, parse_dates=['per_end_date'])
    print(f"    → {len(data['shrs']):,} rows")
    
    # Master ticker (sector info)
    mt_file = list(DATA_DIR.glob('ZACKS_MT_2_*.csv'))[0]
    print(f"  Loading {mt_file.name}...")
    data['mt'] = pd.read_csv(mt_file)
    print(f"    → {len(data['mt']):,} rows")
    
    print("✓ ZACKS data loaded successfully\n")
    return data


def load_price_data(tickers: Optional[List[str]] = None) -> pd.DataFrame:
    """
    Load QUOTEMEDIA price data.
    
    Args:
        tickers: Optional list of tickers to filter (reduces memory)
    
    Returns:
        DataFrame with columns: ticker, date, adj_close, adj_volume
    """
    prices_file = list(DATA_DIR.glob('QUOTEMEDIA_PRICES_*.csv'))[0]
    print(f"Loading {prices_file.name} (this may take a moment...)")
    
    # Use chunked reading for large file
    usecols = ['ticker', 'date', 'adj_close', 'adj_volume']
    
    if tickers is not None:
        # Filter by ticker during load (memory efficient)
        ticker_set = set(tickers)
        chunks = []
        for chunk in pd.read_csv(prices_file, usecols=usecols, parse_dates=['date'], chunksize=1_000_000):
            filtered = chunk[chunk['ticker'].isin(ticker_set)]
            chunks.append(filtered)
        prices = pd.concat(chunks, ignore_index=True)
    else:
        prices = pd.read_csv(prices_file, usecols=usecols, parse_dates=['date'])
    
    print(f"  → {len(prices):,} price records loaded")
    return prices


print("✓ Data loading functions defined")

✓ Data loading functions defined


## 6. Universe Definition (Section 3 of Assignment)

Apply 5 sequential filters to construct the ~1200 ticker universe.

In [46]:
def apply_universe_filters(
    zacks_data: Dict[str, pd.DataFrame],
    start_date: str,
    end_date: str,
) -> pd.DataFrame:
    """
    Apply 5 universe filters per assignment Section 3.
    
    Filters:
    1. Date coverage: prices available for entire period
    2. Market cap: never below $100MM
    3. Debt ratio: > 0.1 somewhere ("more than fleetingly")
    4. Sector: exclude automotive, financial, insurance
    5. Ratio feasibility: all 3 ratios computable
    
    Returns:
        DataFrame with universe tickers and metadata
    """
    print("=" * 80)
    print("APPLYING UNIVERSE FILTERS")
    print("=" * 80)
    
    # Convert dates
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)
    
    # Filter 1: Date coverage
    print("\n[Filter 1] Date coverage check")
    print(f"  Loading sample of price data to check coverage...")
    
    # Load just ticker and date columns for coverage check
    prices_file = list(DATA_DIR.glob('QUOTEMEDIA_PRICES_*.csv'))[0]
    prices_sample = pd.read_csv(prices_file, usecols=['ticker', 'date'], parse_dates=['date'])
    prices_period = prices_sample[
        (prices_sample['date'] >= start_dt) & 
        (prices_sample['date'] <= end_dt)
    ]
    
    # Count trading days per ticker
    total_days = len(prices_period['date'].unique())
    ticker_days = prices_period.groupby('ticker')['date'].nunique()
    
    # Require at least 95% coverage (allow for some holidays/suspensions)
    min_days = int(total_days * 0.95)
    tickers_with_coverage = ticker_days[ticker_days >= min_days].index.tolist()
    
    print(f"  Total trading days in period: {total_days}")
    print(f"  Required coverage: {min_days} days (95%)")
    print(f"  ✓ Passed Filter 1: {len(tickers_with_coverage):,} tickers")
    
    universe = pd.DataFrame({'ticker': tickers_with_coverage})
    
    # Filter 2: Market cap minimum
    print("\n[Filter 2] Market cap minimum ($100MM)")
    mktv = zacks_data['mktv']
    mktv_period = mktv[
        (mktv['per_end_date'] >= start_dt) & 
        (mktv['per_end_date'] <= end_dt)
    ]
    
    # Find minimum market cap per ticker
    min_mktv = mktv_period.groupby('ticker')['mkt_val'].min()
    tickers_mktv_ok = min_mktv[min_mktv >= MIN_MARKET_CAP_MM].index.tolist()
    
    universe = universe[universe['ticker'].isin(tickers_mktv_ok)]
    print(f"  ✓ Passed Filter 2: {len(universe):,} tickers")
    
    # Filter 3: Debt ratio existence
    print(f"\n[Filter 3] Debt ratio > {MIN_DEBT_RATIO} (at least {MIN_DEBT_RATIO_COUNT} quarters)")
    fr = zacks_data['fr']
    fr_period = fr[
        (fr['per_end_date'] >= start_dt) & 
        (fr['per_end_date'] <= end_dt)
    ]
    
    # Count quarters with debt ratio > threshold
    debt_ratio_counts = fr_period[
        fr_period['tot_debt_tot_equity'] > MIN_DEBT_RATIO
    ].groupby('ticker').size()
    
    tickers_debt_ok = debt_ratio_counts[debt_ratio_counts >= MIN_DEBT_RATIO_COUNT].index.tolist()
    
    universe = universe[universe['ticker'].isin(tickers_debt_ok)]
    print(f"  ✓ Passed Filter 3: {len(universe):,} tickers")
    
    # Filter 4: Sector exclusions
    print(f"\n[Filter 4] Sector exclusions: {EXCLUDED_SECTORS}")
    mt = zacks_data['mt']
    
    # Map sector codes (need to check actual column names)
    if 'zacks_sector_code' in mt.columns:
        sector_col = 'zacks_sector_code'
    elif 'zacks_x_sector_desc' in mt.columns:
        sector_col = 'zacks_x_sector_desc'
    else:
        # Try to find any sector column
        sector_cols = [c for c in mt.columns if 'sector' in c.lower()]
        sector_col = sector_cols[0] if sector_cols else None
    
    if sector_col:
        excluded_tickers = mt[
            mt[sector_col].str.contains('|'.join(EXCLUDED_SECTORS), case=False, na=False)
        ]['ticker'].tolist()
        
        universe = universe[~universe['ticker'].isin(excluded_tickers)]
        print(f"  Excluded {len(excluded_tickers):,} tickers from excluded sectors")
    else:
        print(f"  ⚠ Warning: Could not find sector column, skipping sector filter")
    
    print(f"  ✓ Passed Filter 4: {len(universe):,} tickers")
    
    # Filter 5: Ratio feasibility
    print("\n[Filter 5] Ratio feasibility (all 3 ratios computable)")
    
    # Check for required columns in FC
    fc = zacks_data['fc']
    fc_period = fc[
        (fc['per_end_date'] >= start_dt) & 
        (fc['per_end_date'] <= end_dt)
    ]
    
    # Tickers with EPS data (for P/E)
    tickers_with_eps = fc_period[
        fc_period['eps_diluted_net'].notna() | fc_period['basic_net_eps'].notna()
    ]['ticker'].unique()
    
    # Tickers with debt data (for debt ratio and ROI)
    tickers_with_debt = fc_period[
        fc_period['tot_lterm_debt'].notna() | fc_period['net_lterm_debt'].notna()
    ]['ticker'].unique()
    
    # Tickers with ROI data
    tickers_with_roi = fr_period['ret_invst'].notna().groupby(fr_period['ticker']).any()
    tickers_with_roi = tickers_with_roi[tickers_with_roi].index.tolist()
    
    # Intersection: all 3 ratios computable
    feasible_tickers = list(
        set(tickers_with_eps) & 
        set(tickers_with_debt) & 
        set(tickers_with_roi)
    )
    
    universe = universe[universe['ticker'].isin(feasible_tickers)]
    print(f"  ✓ Passed Filter 5: {len(universe):,} tickers")
    
    # Summary
    print("\n" + "=" * 80)
    print(f"FINAL UNIVERSE: {len(universe):,} tickers")
    print("=" * 80)
    
    return universe


print("✓ Universe filter functions defined")

✓ Universe filter functions defined


### Execute Universe Construction

Load data and apply filters to build the ~1200 ticker universe.

In [47]:
# Load ZACKS fundamental data
zacks_data = load_zacks_data()

Loading ZACKS data files...
  Loading ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.csv...
    → 649,883 rows
  Loading ZACKS_FR_2_f40c6a304f87d9f492c1f21839d474e2.csv...
    → 649,883 rows
  Loading ZACKS_MKTV_2_ecb7f768974bbdd26964caefe2fd0378.csv...
    → 1,058,327 rows
  Loading ZACKS_SHRS_2_99db6fa97ac677f3c0d45a9fa9a70196.csv...
    → 1,058,399 rows
  Loading ZACKS_MT_2_5c2afb6368dcc3ed48e1a84279323e63.csv...
    → 38,868 rows
✓ ZACKS data loaded successfully



In [ ]:
# Apply universe filters
universe = apply_universe_filters(zacks_data, START_DATE, END_DATE)

# Display sample
print("\nUniverse sample:")
print(universe.head(10))

# Export universe list
universe_file = RESULTS_DIR / 'universe_tickers.csv'
universe.to_csv(universe_file, index=False)
print(f"\n✓ Universe saved to {universe_file}")

APPLYING UNIVERSE FILTERS

[Filter 1] Date coverage check
  Loading sample of price data to check coverage...


### Load Price Data for Universe

Now that we have the universe, load only the relevant price data.

In [ ]:
# Load prices for universe tickers only (memory efficient)
universe_tickers = universe['ticker'].tolist()
prices = load_price_data(tickers=universe_tickers)

# Filter to date range
prices = prices[
    (prices['date'] >= START_DATE) & 
    (prices['date'] <= END_DATE)
].sort_values(['ticker', 'date']).reset_index(drop=True)

print(f"\nPrice data shape: {prices.shape}")
print(f"Date range: {prices['date'].min()} to {prices['date'].max()}")
print(f"Unique tickers: {prices['ticker'].nunique()}")

# Display sample
print("\nPrice data sample:")
print(prices.head(10))

Loading QUOTEMEDIA_PRICES_247f636d651d8ef83d8ca1e756cf5ee4.csv (this may take a moment...)
  → 10,530,600 price records loaded

Price data shape: (2295752, 4)
Date range: 2018-01-02 00:00:00 to 2023-06-30 00:00:00
Unique tickers: 1661

Price data sample:
  ticker       date  adj_close  adj_volume
0      A 2018-01-02    64.3125   1047830.0
1      A 2018-01-03    65.9488   1698899.0
2      A 2018-01-04    65.4541   2231534.0
3      A 2018-01-05    66.5006   1632512.0
4      A 2018-01-08    66.6433   1613911.0
5      A 2018-01-09    68.2797   2666711.0
6      A 2018-01-10    67.3473   2957184.0
7      A 2018-01-11    67.3569   1511134.0
8      A 2018-01-12    68.2416   1448155.0
9      A 2018-01-16    67.7659   1703398.0


---

## 7. Financial Ratio Computation (Section 4 of Assignment)

Implement filing-date-aware ratio computation with daily market cap adjustments.

### 7.1 Import Validated Ratio Calculator

Import the production-ready `calculate_financial_ratios` function from ratio_calculator.py.

In [ ]:
# Import validated ratio calculator
import sys
sys.path.insert(0, str(Path('.').resolve()))
from ratio_calculator import calculate_financial_ratios

print("✓ Imported calculate_financial_ratios from ratio_calculator.py")

✓ Imported calculate_financial_ratios from ratio_calculator.py


### 7.2 Vectorized Daily Ratio Computation

Compute all three ratios for the entire universe with daily updates based on price changes.

In [ ]:
def compute_all_ratios(
    tickers: List[str],
    fc: pd.DataFrame,
    fr: pd.DataFrame,
    mktv: pd.DataFrame,
    shrs: pd.DataFrame,
    prices: pd.DataFrame,
    start_date: str,
    end_date: str,
    progress_interval: int = 100
) -> pd.DataFrame:
    """
    Compute all three financial ratios for every ticker on every trading day.
    
    Uses filing-date-aware computation: ratios are forward-filled from filing_date
    to the next filing_date, with daily price adjustments.
    
    Args:
        tickers: List of universe tickers
        fc: Financial Condition data (ZACKS_FC)
        fr: Financial Ratios data (ZACKS_FR)
        mktv: Market Value data (ZACKS_MKTV)
        shrs: Shares Outstanding data (ZACKS_SHRS)
        prices: Daily price data (QUOTEMEDIA_PRICES)
        start_date: Start of analysis period
        end_date: End of analysis period
        progress_interval: Print progress every N tickers
    
    Returns:
        DataFrame with columns: ticker, date, debt_mktcap, roi, pe
    """
    print("=" * 80)
    print("COMPUTING FINANCIAL RATIOS FOR ALL TICKERS")
    print("=" * 80)
    
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)
    
    # Preprocess: index fundamental data by ticker for fast lookup
    fc_by_ticker = {t: g.sort_values('filing_date') for t, g in fc.groupby('ticker')}
    fr_by_ticker = {t: g.sort_values('per_end_date') for t, g in fr.groupby('ticker')}
    mktv_by_ticker = {t: g.sort_values('per_end_date') for t, g in mktv.groupby('ticker')}
    shrs_by_ticker = {t: g.sort_values('per_end_date') for t, g in shrs.groupby('ticker')}
    prices_by_ticker = {t: g.sort_values('date') for t, g in prices.groupby('ticker')}
    
    all_ratios = []
    failed_tickers = []
    
    for i, ticker in enumerate(tickers):
        if (i + 1) % progress_interval == 0:
            print(f"  Processing ticker {i+1}/{len(tickers)}: {ticker}")
        
        # Get ticker data
        ticker_fc = fc_by_ticker.get(ticker)
        ticker_fr = fr_by_ticker.get(ticker)
        ticker_mktv = mktv_by_ticker.get(ticker)
        ticker_shrs = shrs_by_ticker.get(ticker)
        ticker_prices = prices_by_ticker.get(ticker)
        
        # Skip if missing data
        if ticker_fc is None or ticker_fr is None or ticker_mktv is None or ticker_shrs is None or ticker_prices is None:
            failed_tickers.append((ticker, "missing_data"))
            continue
        
        if len(ticker_fc) == 0 or len(ticker_prices) == 0:
            failed_tickers.append((ticker, "empty_data"))
            continue
        
        # Filter to date range
        ticker_fc = ticker_fc[(ticker_fc['filing_date'] >= start_dt - pd.Timedelta(days=365)) & 
                               (ticker_fc['filing_date'] <= end_dt)]
        ticker_prices = ticker_prices[(ticker_prices['date'] >= start_dt) & 
                                       (ticker_prices['date'] <= end_dt)]
        
        if len(ticker_fc) == 0 or len(ticker_prices) == 0:
            failed_tickers.append((ticker, "no_data_in_range"))
            continue
        
        try:
            # Build daily ratio series
            ticker_ratios = []
            
            # Get all unique filing dates
            filing_dates = ticker_fc['filing_date'].sort_values().unique()
            
            for j, filing_date in enumerate(filing_dates):
                # Get fundamental data for this filing
                fc_row = ticker_fc[ticker_fc['filing_date'] == filing_date].iloc[0]
                per_end = fc_row['per_end_date']
                
                # Find matching FR, MKTV, SHRS data
                fr_match = ticker_fr[ticker_fr['per_end_date'] == per_end]
                mktv_match = ticker_mktv[ticker_mktv['per_end_date'] == per_end]
                shrs_match = ticker_shrs[ticker_shrs['per_end_date'] == per_end]
                
                if len(fr_match) == 0 or len(mktv_match) == 0 or len(shrs_match) == 0:
                    continue
                
                fr_row = fr_match.iloc[0]
                mktv_row = mktv_match.iloc[0]
                shrs_row = shrs_match.iloc[0]
                
                # Determine date range this filing applies to
                next_filing = filing_dates[j + 1] if j + 1 < len(filing_dates) else end_dt + pd.Timedelta(days=1)
                
                # Get prices in this range
                filing_date_ts = pd.Timestamp(filing_date)
                next_filing_ts = pd.Timestamp(next_filing)
                
                range_prices = ticker_prices[
                    (ticker_prices['date'] >= filing_date_ts) & 
                    (ticker_prices['date'] < next_filing_ts)
                ]
                
                if len(range_prices) == 0:
                    continue
                
                # Compute ratios for each trading day
                for _, price_row in range_prices.iterrows():
                    try:
                        ratios = calculate_financial_ratios(
                            ticker=ticker,
                            per_end_date=per_end,
                            current_date=price_row['date'],
                            fc_data=fc_row,
                            fr_data=fr_row,
                            mktv_data=mktv_row,
                            shrs_data=shrs_row,
                            prices_data=ticker_prices
                        )
                        
                        ticker_ratios.append({
                            'ticker': ticker,
                            'date': price_row['date'],
                            'debt_mktcap': ratios['debt_mktcap'],
                            'roi': ratios['roi'],
                            'pe': ratios['pe'],
                            'mkt_cap': ratios['mkt_cap']
                        })
                    except Exception as e:
                        # Skip days with computation errors
                        continue
            
            if ticker_ratios:
                all_ratios.extend(ticker_ratios)
        
        except Exception as e:
            failed_tickers.append((ticker, str(e)[:50]))
            continue
    
    # Combine all ratios
    ratios_df = pd.DataFrame(all_ratios)
    
    print(f"\n✓ Computed ratios for {ratios_df['ticker'].nunique():,} tickers")
    print(f"  Total daily observations: {len(ratios_df):,}")
    print(f"  Failed tickers: {len(failed_tickers):,}")
    
    if failed_tickers[:5]:
        print(f"  Sample failures: {failed_tickers[:5]}")
    
    return ratios_df


print("✓ compute_all_ratios function defined")

✓ compute_all_ratios function defined


In [ ]:
# Compute ratios for the entire universe
# This cell takes several minutes to run

ratios_df = compute_all_ratios(
    tickers=universe_tickers,
    fc=zacks_data['fc'],
    fr=zacks_data['fr'],
    mktv=zacks_data['mktv'],
    shrs=zacks_data['shrs'],
    prices=prices,
    start_date=START_DATE,
    end_date=END_DATE,
    progress_interval=100
)

# Display sample
print("\nRatios sample:")
print(ratios_df.head(10))
print(f"\nDate range: {ratios_df['date'].min()} to {ratios_df['date'].max()}")

COMPUTING FINANCIAL RATIOS FOR ALL TICKERS
  Processing ticker 100/1661: AQN
  Processing ticker 200/1661: BKNG
  Processing ticker 300/1661: CLH
  Processing ticker 400/1661: DAL
  Processing ticker 500/1661: ESE
  Processing ticker 600/1661: GHC
  Processing ticker 700/1661: HTHT
  Processing ticker 800/1661: KGC
  Processing ticker 900/1661: MCFT
  Processing ticker 1000/1661: NDSN
  Processing ticker 1100/1661: OSG
  Processing ticker 1200/1661: PXD
  Processing ticker 1300/1661: SENEA
  Processing ticker 1400/1661: SXI


KeyboardInterrupt: 

### 7.3 Combined Scoring Method

Create a combined signal using z-score normalized ratios with equal weighting.

**Signal Interpretation:**
- **Debt/MktCap**: Lower is better (less leverage) → invert for ranking
- **ROI**: Higher is better (more efficient)
- **P/E**: Lower is better (cheaper valuation) → invert for ranking

In [ ]:
def compute_combined_score(ratios_df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute combined score using z-score normalization and equal weighting.
    
    For each date:
    1. Z-score normalize each ratio across all tickers
    2. Invert signals where lower is better (debt_mktcap, pe)
    3. Average the z-scores to get combined score
    
    Args:
        ratios_df: DataFrame with ticker, date, debt_mktcap, roi, pe
    
    Returns:
        DataFrame with additional columns: z_debt_mktcap, z_roi, z_pe, combined_score
    """
    print("Computing combined scores...")
    
    df = ratios_df.copy()
    
    # Handle infinities and extreme outliers by winsorizing
    for col in ['debt_mktcap', 'roi', 'pe']:
        # Replace infinities
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
        # Winsorize at 1st and 99th percentiles
        lower = df[col].quantile(0.01)
        upper = df[col].quantile(0.99)
        df[col] = df[col].clip(lower, upper)
    
    # Z-score normalize within each date
    def zscore_group(group):
        for col in ['debt_mktcap', 'roi', 'pe']:
            mean = group[col].mean()
            std = group[col].std()
            if std > 0:
                group[f'z_{col}'] = (group[col] - mean) / std
            else:
                group[f'z_{col}'] = 0
        return group
    
    df = df.groupby('date', group_keys=False).apply(zscore_group)
    
    # Invert signals where lower is better
    # debt_mktcap: lower is better → negate z-score
    # pe: lower is better → negate z-score
    # roi: higher is better → keep as is
    df['z_debt_mktcap_adj'] = -df['z_debt_mktcap']
    df['z_pe_adj'] = -df['z_pe']
    df['z_roi_adj'] = df['z_roi']
    
    # Combined score: equal-weighted average
    df['combined_score'] = (df['z_debt_mktcap_adj'] + df['z_roi_adj'] + df['z_pe_adj']) / 3
    
    print(f"✓ Combined scores computed for {len(df):,} observations")
    
    return df


# Compute combined scores
ratios_df = compute_combined_score(ratios_df)

print("\nScore summary statistics:")
print(ratios_df[['debt_mktcap', 'roi', 'pe', 'combined_score']].describe())

### 7.4 Ratio Changes Computation

Per assignment Section 5: Compute ratio changes (Δratio = ratio_t - ratio_{t-1}) as an alternative signal.
This tests whether **changes** in fundamentals are more predictive than **levels**.

In [ ]:
def compute_ratio_changes(ratios_df: pd.DataFrame, lookback_days: int = 21) -> pd.DataFrame:
    """
    Compute ratio changes from lookback period (approximately 1 month).
    
    Δratio = ratio_today - ratio_lookback_days_ago
    
    Args:
        ratios_df: DataFrame with daily ratios
        lookback_days: Number of trading days for change calculation
    
    Returns:
        DataFrame with additional change columns
    """
    print(f"Computing ratio changes (lookback={lookback_days} days)...")
    
    df = ratios_df.copy().sort_values(['ticker', 'date'])
    
    # Compute lagged values for each ticker
    for col in ['debt_mktcap', 'roi', 'pe', 'combined_score']:
        df[f'{col}_lag'] = df.groupby('ticker')[col].shift(lookback_days)
        df[f'd_{col}'] = df[col] - df[f'{col}_lag']
    
    # Drop rows without valid changes (beginning of series)
    valid_mask = df['d_debt_mktcap'].notna()
    df_changes = df[valid_mask].copy()
    
    print(f"✓ Computed changes for {len(df_changes):,} observations")
    print(f"  Dropped {(~valid_mask).sum():,} observations without lookback data")
    
    return df_changes


# Compute ratio changes
ratios_with_changes = compute_ratio_changes(ratios_df, lookback_days=21)

print("\nChange statistics:")
print(ratios_with_changes[['d_debt_mktcap', 'd_roi', 'd_pe', 'd_combined_score']].describe())

In [ ]:
# Save computed ratios for later use
ratios_file = RESULTS_DIR / 'computed_ratios.parquet'
ratios_df.to_parquet(ratios_file, index=False)
print(f"✓ Saved ratios to {ratios_file}")

changes_file = RESULTS_DIR / 'ratio_changes.parquet'
ratios_with_changes.to_parquet(changes_file, index=False)
print(f"✓ Saved ratio changes to {changes_file}")

---

## 8. Backtesting Engine

Implements the quantile long-short strategy with:
- Monthly and weekly rebalancing
- Quantile-based portfolio construction (top/bottom decile)
- Position sizing variations (base, vigintile doubling, vigintile halving)
- Funding cost calculations

### 8.1 Strategy Configuration

In [ ]:
@dataclass
class StrategyConfig:
    """Configuration for a single strategy variant."""
    name: str
    signal_col: str          # Column to rank on (e.g., 'debt_mktcap', 'combined_score', 'd_roi')
    rebalance_freq: str      # 'M' for monthly, 'W' for weekly
    sizing_method: str       # 'equal', 'vigintile_double', 'vigintile_half'
    long_quantile: float     # Percentile threshold for long (e.g., 0.90)
    short_quantile: float    # Percentile threshold for short (e.g., 0.10)
    invert_signal: bool      # True if lower signal value is better (for long)
    use_changes: bool = False  # True to use ratio changes instead of levels


# Define all 24+ strategy variants
STRATEGY_CONFIGS = []

# Scoring methods
scoring_methods = [
    ('debt_mktcap', True, False),   # Lower debt ratio is better for longs
    ('roi', False, False),           # Higher ROI is better for longs
    ('pe', True, False),             # Lower P/E is better for longs
    ('combined_score', False, False),  # Higher combined score is better
    # Ratio changes variants
    ('d_debt_mktcap', True, True),   # Improving (decreasing) debt ratio
    ('d_roi', False, True),          # Improving ROI
    ('d_combined_score', False, True),  # Improving combined score
]

# Rebalance frequencies
frequencies = ['M', 'W']

# Sizing methods
sizing_methods = ['equal', 'vigintile_double', 'vigintile_half']

# Generate all combinations
for signal_col, invert, use_changes in scoring_methods:
    for freq in frequencies:
        for sizing in sizing_methods:
            change_suffix = "_chg" if use_changes else ""
            name = f"{signal_col}{change_suffix}_{freq}_{sizing}"
            
            config = StrategyConfig(
                name=name,
                signal_col=signal_col,
                rebalance_freq=freq,
                sizing_method=sizing,
                long_quantile=0.90,
                short_quantile=0.10,
                invert_signal=invert,
                use_changes=use_changes
            )
            STRATEGY_CONFIGS.append(config)

print(f"✓ Defined {len(STRATEGY_CONFIGS)} strategy configurations")
print("\nSample configurations:")
for config in STRATEGY_CONFIGS[:6]:
    print(f"  {config.name}: signal={config.signal_col}, freq={config.rebalance_freq}, sizing={config.sizing_method}")

### 8.2 Portfolio Construction

Build long-short portfolios based on quantile rankings.

In [ ]:
def get_rebalance_dates(
    start_date: str,
    end_date: str,
    freq: str = 'M'
) -> List[pd.Timestamp]:
    """
    Generate rebalance dates for the strategy.
    
    Args:
        start_date: Start of analysis period
        end_date: End of analysis period
        freq: 'M' for monthly (month-end), 'W' for weekly (Friday)
    
    Returns:
        List of rebalance dates
    """
    dates = pd.date_range(start=start_date, end=end_date, freq=freq)
    
    if freq == 'W':
        # Weekly: use Friday
        dates = pd.date_range(start=start_date, end=end_date, freq='W-FRI')
    elif freq == 'M':
        # Monthly: use month-end
        dates = pd.date_range(start=start_date, end=end_date, freq='ME')
    
    return dates.tolist()


def construct_portfolio(
    ratios_on_date: pd.DataFrame,
    signal_col: str,
    long_quantile: float,
    short_quantile: float,
    invert_signal: bool,
    sizing_method: str
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Construct long and short portfolios based on signal ranking.
    
    Args:
        ratios_on_date: DataFrame with ratios for a single date
        signal_col: Column to rank on
        long_quantile: Percentile threshold for long positions
        short_quantile: Percentile threshold for short positions
        invert_signal: If True, lower values are better for longs
        sizing_method: 'equal', 'vigintile_double', 'vigintile_half'
    
    Returns:
        Tuple of (long_portfolio, short_portfolio) DataFrames with weights
    """
    df = ratios_on_date.dropna(subset=[signal_col]).copy()
    
    if len(df) < 20:  # Need enough stocks for decile ranking
        return pd.DataFrame(), pd.DataFrame()
    
    # Rank stocks (lower rank = lower signal value)
    df['rank'] = df[signal_col].rank(pct=True)
    
    # Determine long and short positions based on signal direction
    if invert_signal:
        # Lower signal is better → long bottom decile, short top decile
        long_mask = df['rank'] <= short_quantile  # Bottom 10%
        short_mask = df['rank'] >= long_quantile  # Top 10%
    else:
        # Higher signal is better → long top decile, short bottom decile
        long_mask = df['rank'] >= long_quantile   # Top 10%
        short_mask = df['rank'] <= short_quantile # Bottom 10%
    
    long_df = df[long_mask].copy()
    short_df = df[short_mask].copy()
    
    # Apply position sizing
    def assign_weights(portfolio_df: pd.DataFrame, is_long: bool) -> pd.DataFrame:
        if len(portfolio_df) == 0:
            return portfolio_df
        
        # Base weight: equal weight within decile
        n = len(portfolio_df)
        base_weight = 1.0 / n
        
        if sizing_method == 'equal':
            portfolio_df['weight'] = base_weight
        
        elif sizing_method == 'vigintile_double':
            # 2x weight for top/bottom 5% (vigintile)
            portfolio_df['weight'] = base_weight
            if invert_signal:
                # Lower rank is more extreme
                extreme_mask = portfolio_df['rank'] <= 0.05 if is_long else portfolio_df['rank'] >= 0.95
            else:
                extreme_mask = portfolio_df['rank'] >= 0.95 if is_long else portfolio_df['rank'] <= 0.05
            portfolio_df.loc[extreme_mask, 'weight'] = base_weight * 2
            # Renormalize
            portfolio_df['weight'] = portfolio_df['weight'] / portfolio_df['weight'].sum()
        
        elif sizing_method == 'vigintile_half':
            # 0.5x weight for top/bottom 5% (vigintile) - distrust outliers
            portfolio_df['weight'] = base_weight
            if invert_signal:
                extreme_mask = portfolio_df['rank'] <= 0.05 if is_long else portfolio_df['rank'] >= 0.95
            else:
                extreme_mask = portfolio_df['rank'] >= 0.95 if is_long else portfolio_df['rank'] <= 0.05
            portfolio_df.loc[extreme_mask, 'weight'] = base_weight * 0.5
            # Renormalize
            portfolio_df['weight'] = portfolio_df['weight'] / portfolio_df['weight'].sum()
        
        return portfolio_df
    
    long_df = assign_weights(long_df, is_long=True)
    short_df = assign_weights(short_df, is_long=False)
    
    return long_df, short_df


print("✓ Portfolio construction functions defined")

### 8.3 Backtesting Engine

Core backtest loop with P&L tracking, funding costs, and trade recording.

In [ ]:
def run_backtest(
    config: StrategyConfig,
    ratios_df: pd.DataFrame,
    prices: pd.DataFrame,
    initial_capital: float = 1_000_000,
    funding_rate: float = 0.02,
    repo_spread: float = 0.01
) -> BacktestResult:
    """
    Run backtest for a single strategy configuration.
    
    Args:
        config: Strategy configuration
        ratios_df: DataFrame with daily ratios
        prices: Price data (must include all tickers in ratios)
        initial_capital: Starting capital
        funding_rate: Annual funding rate for longs
        repo_spread: Spread below funding for short rebate
    
    Returns:
        BacktestResult with equity curve, trades, and metrics
    """
    # Use the appropriate data source
    data = ratios_df.copy()
    
    # If using changes, filter to valid change data
    if config.use_changes and config.signal_col not in data.columns:
        print(f"  Warning: {config.signal_col} not in data, skipping")
        return None
    
    # Get rebalance dates
    rebalance_dates = get_rebalance_dates(START_DATE, END_DATE, config.rebalance_freq)
    
    # Get all trading days
    all_dates = sorted(data['date'].unique())
    
    # Initialize tracking
    capital = initial_capital
    equity_history = []
    trades = []
    current_positions = {}  # ticker -> (shares, entry_price, entry_date, side)
    
    # Precompute price lookup
    prices_pivot = prices.pivot(index='date', columns='ticker', values='adj_close')
    
    # Daily funding/repo rates
    daily_funding = funding_rate / 252
    daily_repo = (funding_rate - repo_spread) / 252
    
    # Track gross notional for capital sizing
    first_gross_notional = None
    
    for i, date in enumerate(all_dates):
        date_ts = pd.Timestamp(date)
        
        # Get today's prices
        if date_ts not in prices_pivot.index:
            continue
        today_prices = prices_pivot.loc[date_ts]
        
        # Update position values
        long_value = 0
        short_value = 0
        
        for ticker, (shares, entry_price, entry_date, side) in list(current_positions.items()):
            if ticker in today_prices and not pd.isna(today_prices[ticker]):
                current_price = today_prices[ticker]
                if side == 'long':
                    long_value += shares * current_price
                else:
                    # Short: value is entry_value - (current_price - entry_price) * shares
                    entry_value = shares * entry_price
                    pnl = (entry_price - current_price) * shares
                    short_value += entry_value + pnl
        
        # Apply daily funding costs
        funding_cost = long_value * daily_funding
        repo_rebate = short_value * daily_repo * 0.5  # Approximate rebate
        
        # Calculate equity
        cash = capital - long_value - short_value  # Simplified
        equity = long_value + short_value + capital  # Mark-to-market
        
        equity_history.append({'date': date_ts, 'equity': equity})
        
        # Check for rebalance
        is_rebalance = any(abs((date_ts - rd).days) <= 2 for rd in rebalance_dates)
        
        if is_rebalance:
            # Get ratios for this date
            date_ratios = data[data['date'] == date_ts]
            
            if len(date_ratios) == 0:
                # Try nearest date within 5 days
                nearby = data[(data['date'] >= date_ts - pd.Timedelta(days=5)) & 
                              (data['date'] <= date_ts + pd.Timedelta(days=5))]
                if len(nearby) > 0:
                    nearest_date = nearby['date'].max()
                    date_ratios = data[data['date'] == nearest_date]
            
            if len(date_ratios) < 20:
                continue
            
            # Construct new portfolios
            long_port, short_port = construct_portfolio(
                date_ratios,
                config.signal_col,
                config.long_quantile,
                config.short_quantile,
                config.invert_signal,
                config.sizing_method
            )
            
            if len(long_port) == 0 and len(short_port) == 0:
                continue
            
            # Calculate target notional
            target_long_notional = capital * 0.5  # 50% long
            target_short_notional = capital * 0.5  # 50% short
            
            if first_gross_notional is None:
                first_gross_notional = target_long_notional + target_short_notional
            
            # Close existing positions (record trades)
            for ticker, (shares, entry_price, entry_date, side) in current_positions.items():
                if ticker in today_prices and not pd.isna(today_prices[ticker]):
                    exit_price = today_prices[ticker]
                    if side == 'long':
                        pnl = (exit_price - entry_price) * shares
                    else:
                        pnl = (entry_price - exit_price) * shares
                    
                    trades.append(Trade(
                        ticker=ticker,
                        side=PositionSide.LONG if side == 'long' else PositionSide.SHORT,
                        entry_time=entry_date,
                        exit_time=date_ts,
                        entry_price=entry_price,
                        exit_price=exit_price,
                        shares=shares,
                        pnl=pnl,
                        exit_reason=ExitReason.REBALANCE
                    ))
            
            # Open new positions
            current_positions = {}
            
            # Long positions
            for _, row in long_port.iterrows():
                ticker = row['ticker']
                if ticker in today_prices and not pd.isna(today_prices[ticker]):
                    price = today_prices[ticker]
                    notional = target_long_notional * row['weight']
                    shares = notional / price
                    current_positions[ticker] = (shares, price, date_ts, 'long')
            
            # Short positions
            for _, row in short_port.iterrows():
                ticker = row['ticker']
                if ticker in today_prices and not pd.isna(today_prices[ticker]):
                    price = today_prices[ticker]
                    notional = target_short_notional * row['weight']
                    shares = notional / price
                    current_positions[ticker] = (shares, price, date_ts, 'short')
    
    # Create equity curve
    equity_df = pd.DataFrame(equity_history)
    if len(equity_df) == 0:
        return None
    
    equity_df = equity_df.set_index('date')['equity']
    
    # Calculate metrics
    total_return = (equity_df.iloc[-1] - initial_capital) / initial_capital
    sharpe = calculate_sharpe_ratio(equity_df)
    max_dd = calculate_max_drawdown(equity_df)
    win_rate = calculate_win_rate(trades)
    
    return BacktestResult(
        trades=trades,
        equity_curve=equity_df,
        final_capital=equity_df.iloc[-1],
        total_return=total_return,
        sharpe_ratio=sharpe,
        max_drawdown=max_dd,
        win_rate=win_rate,
        num_trades=len(trades),
        num_rebalances=len([t for t in trades if t.exit_reason == ExitReason.REBALANCE]) // 2,
        params={
            'name': config.name,
            'signal': config.signal_col,
            'freq': config.rebalance_freq,
            'sizing': config.sizing_method
        }
    )


print("✓ Backtest engine defined")

### 8.4 Run All Strategy Variants

Execute backtests for all 42 strategy configurations (7 signals × 2 frequencies × 3 sizing methods).

In [ ]:
# Run all backtests
# This cell takes several minutes to run

print("=" * 80)
print("RUNNING ALL BACKTESTS")
print("=" * 80)

# Prepare data sources
levels_data = ratios_df  # Absolute ratio levels
changes_data = ratios_with_changes  # Ratio changes

all_results = {}

for i, config in enumerate(STRATEGY_CONFIGS):
    print(f"\n[{i+1}/{len(STRATEGY_CONFIGS)}] Running: {config.name}")
    
    # Select appropriate data source
    if config.use_changes:
        data = changes_data
    else:
        data = levels_data
    
    # Run backtest
    result = run_backtest(
        config=config,
        ratios_df=data,
        prices=prices,
        initial_capital=1_000_000,
        funding_rate=FUNDING_RATE,
        repo_spread=REPO_SPREAD
    )
    
    if result is not None:
        all_results[config.name] = result
        print(f"  ✓ Sharpe: {result.sharpe_ratio:.3f}, Return: {result.total_return:.2%}, MaxDD: {result.max_drawdown:.2%}")
    else:
        print(f"  ✗ Backtest failed or insufficient data")

print("\n" + "=" * 80)
print(f"COMPLETED: {len(all_results)} successful backtests out of {len(STRATEGY_CONFIGS)}")
print("=" * 80)

---

## 9. Performance Analysis

Comprehensive performance metrics for all strategy variants.

### 9.1 Results Summary Table

In [ ]:
def create_results_summary(results: Dict[str, BacktestResult]) -> pd.DataFrame:
    """
    Create summary table of all backtest results.
    
    Args:
        results: Dictionary of strategy name -> BacktestResult
    
    Returns:
        DataFrame with performance metrics for each strategy
    """
    rows = []
    
    for name, result in results.items():
        # Calculate additional metrics
        returns = result.equity_curve.pct_change().dropna()
        
        # Calmar ratio
        calmar = calculate_calmar_ratio(result.total_return, result.max_drawdown)
        
        # Tail risk (VaR)
        var_1pct = calculate_tail_risk(returns, 0.01) if len(returns) > 10 else 0
        var_5pct = calculate_tail_risk(returns, 0.05) if len(returns) > 10 else 0
        
        # P&L to notional ratio
        total_pnl = result.final_capital - 1_000_000
        traded_notional = sum(abs(t.shares * t.entry_price) for t in result.trades)
        pnl_notional_ratio = total_pnl / traded_notional if traded_notional > 0 else 0
        
        rows.append({
            'Strategy': name,
            'Signal': result.params['signal'],
            'Frequency': result.params['freq'],
            'Sizing': result.params['sizing'],
            'Total Return': result.total_return,
            'Sharpe Ratio': result.sharpe_ratio,
            'Max Drawdown': result.max_drawdown,
            'Win Rate': result.win_rate,
            'Calmar Ratio': calmar,
            'VaR 1%': var_1pct,
            'VaR 5%': var_5pct,
            'PnL/Notional': pnl_notional_ratio,
            'Num Trades': result.num_trades,
            'Final Capital': result.final_capital
        })
    
    df = pd.DataFrame(rows)
    df = df.sort_values('Sharpe Ratio', ascending=False)
    
    return df


# Create results summary
results_summary = create_results_summary(all_results)

print("=" * 100)
print("PERFORMANCE SUMMARY (Top 15 by Sharpe Ratio)")
print("=" * 100)
print(results_summary.head(15).to_string(index=False))

# Save results
summary_file = RESULTS_DIR / 'backtest_results.csv'
results_summary.to_csv(summary_file, index=False)
print(f"\n✓ Results saved to {summary_file}")

### 9.2 Ratio Levels vs Ratio Changes Comparison

Compare the performance of strategies using absolute ratio levels vs ratio changes (Δratio).

In [ ]:
# Compare levels vs changes
levels_strategies = results_summary[~results_summary['Signal'].str.startswith('d_')]
changes_strategies = results_summary[results_summary['Signal'].str.startswith('d_')]

print("=" * 80)
print("LEVELS vs CHANGES COMPARISON")
print("=" * 80)

print("\n### Absolute Ratio Levels Strategies ###")
print(f"Count: {len(levels_strategies)}")
print(f"Avg Sharpe: {levels_strategies['Sharpe Ratio'].mean():.3f}")
print(f"Avg Return: {levels_strategies['Total Return'].mean():.2%}")
print(f"Avg Max DD: {levels_strategies['Max Drawdown'].mean():.2%}")

print("\n### Ratio Changes (Δ) Strategies ###")
print(f"Count: {len(changes_strategies)}")
if len(changes_strategies) > 0:
    print(f"Avg Sharpe: {changes_strategies['Sharpe Ratio'].mean():.3f}")
    print(f"Avg Return: {changes_strategies['Total Return'].mean():.2%}")
    print(f"Avg Max DD: {changes_strategies['Max Drawdown'].mean():.2%}")
else:
    print("No change-based strategies completed successfully.")

# Best performers from each category
print("\n### Best Level-Based Strategy ###")
print(levels_strategies.head(1)[['Strategy', 'Sharpe Ratio', 'Total Return', 'Max Drawdown']].to_string(index=False))

if len(changes_strategies) > 0:
    print("\n### Best Change-Based Strategy ###")
    print(changes_strategies.head(1)[['Strategy', 'Sharpe Ratio', 'Total Return', 'Max Drawdown']].to_string(index=False))

### 9.3 Signal Comparison

Compare performance across the four main scoring methods.

In [ ]:
# Group by signal type (excluding changes)
main_signals = ['debt_mktcap', 'roi', 'pe', 'combined_score']
signal_comparison = levels_strategies[levels_strategies['Signal'].isin(main_signals)].groupby('Signal').agg({
    'Sharpe Ratio': ['mean', 'std', 'max'],
    'Total Return': ['mean', 'max'],
    'Max Drawdown': ['mean', 'min'],
    'Win Rate': 'mean'
}).round(4)

print("=" * 80)
print("SIGNAL COMPARISON (Levels Only)")
print("=" * 80)
print(signal_comparison)

# Rank signals by average Sharpe
signal_ranking = levels_strategies[levels_strategies['Signal'].isin(main_signals)].groupby('Signal')['Sharpe Ratio'].mean().sort_values(ascending=False)
print("\n### Signal Ranking by Average Sharpe ###")
for i, (signal, sharpe) in enumerate(signal_ranking.items(), 1):
    print(f"{i}. {signal}: {sharpe:.3f}")

### 9.4 Position Sizing Comparison

Evaluate the impact of vigintile doubling vs halving vs equal weighting.

In [ ]:
# Group by sizing method
sizing_comparison = results_summary.groupby('Sizing').agg({
    'Sharpe Ratio': ['mean', 'std', 'max'],
    'Total Return': ['mean', 'max'],
    'Max Drawdown': ['mean', 'min'],
    'Win Rate': 'mean'
}).round(4)

print("=" * 80)
print("POSITION SIZING COMPARISON")
print("=" * 80)
print(sizing_comparison)

print("\n### Interpretation ###")
print("""
- **equal**: Baseline equal-weight within decile
- **vigintile_double**: 2x weight for extreme quintile (most attractive/unattractive)
  Hypothesis: Extreme values have stronger signal
- **vigintile_half**: 0.5x weight for extreme quintile
  Hypothesis: Extreme values are untrustworthy outliers
""")

### 9.5 Rebalancing Frequency Comparison

Monthly vs weekly rebalancing.

In [ ]:
# Group by rebalancing frequency
freq_comparison = results_summary.groupby('Frequency').agg({
    'Sharpe Ratio': ['mean', 'std', 'max'],
    'Total Return': ['mean', 'max'],
    'Max Drawdown': ['mean', 'min'],
    'Num Trades': 'mean'
}).round(4)

print("=" * 80)
print("REBALANCING FREQUENCY COMPARISON")
print("=" * 80)
print(freq_comparison)

print("\n### Interpretation ###")
print("""
- **M (Monthly)**: Lower turnover, lower transaction costs, potential signal decay
- **W (Weekly)**: Higher turnover, captures signal changes faster, higher transaction costs
""")

---

## 10. Visualizations

Charts and figures for strategy analysis.

### 10.1 Equity Curves by Signal Type

In [ ]:
def plot_equity_curves_by_signal(results: Dict[str, BacktestResult], sizing: str = 'equal', freq: str = 'M'):
    """
    Plot equity curves for each signal type (holding sizing and frequency constant).
    """
    fig, ax = plt.subplots(figsize=(14, 7))
    
    colors = {'debt_mktcap': '#1f77b4', 'roi': '#ff7f0e', 'pe': '#2ca02c', 'combined_score': '#d62728'}
    
    for name, result in results.items():
        if result.params['sizing'] == sizing and result.params['freq'] == freq:
            signal = result.params['signal']
            if signal in colors and not signal.startswith('d_'):
                equity_normalized = result.equity_curve / result.equity_curve.iloc[0]
                ax.plot(equity_normalized.index, equity_normalized.values, 
                       label=signal, color=colors[signal], linewidth=1.5)
    
    ax.set_title(f'Equity Curves by Signal Type ({freq} rebalancing, {sizing} sizing)', fontsize=14)
    ax.set_xlabel('Date')
    ax.set_ylabel('Normalized Equity (Starting = 1.0)')
    ax.legend(loc='upper left')
    ax.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'equity_curves_by_signal.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved to {PLOTS_DIR / 'equity_curves_by_signal.png'}")


# Plot equity curves
plot_equity_curves_by_signal(all_results, sizing='equal', freq='M')

### 10.2 Drawdown Analysis

In [ ]:
def plot_drawdowns(results: Dict[str, BacktestResult], sizing: str = 'equal', freq: str = 'M'):
    """
    Plot drawdown curves for each signal type.
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    colors = {'debt_mktcap': '#1f77b4', 'roi': '#ff7f0e', 'pe': '#2ca02c', 'combined_score': '#d62728'}
    
    for name, result in results.items():
        if result.params['sizing'] == sizing and result.params['freq'] == freq:
            signal = result.params['signal']
            if signal in colors and not signal.startswith('d_'):
                equity = result.equity_curve
                running_max = equity.cummax()
                drawdown = (equity - running_max) / running_max
                ax.fill_between(drawdown.index, drawdown.values, 0, 
                               alpha=0.3, label=signal, color=colors[signal])
                ax.plot(drawdown.index, drawdown.values, color=colors[signal], linewidth=0.8)
    
    ax.set_title(f'Drawdown Analysis ({freq} rebalancing, {sizing} sizing)', fontsize=14)
    ax.set_xlabel('Date')
    ax.set_ylabel('Drawdown (%)')
    ax.legend(loc='lower left')
    ax.set_ylim(bottom=ax.get_ylim()[0] * 1.1)
    ax.grid(True, alpha=0.3)
    
    # Format y-axis as percentage
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'drawdown_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved to {PLOTS_DIR / 'drawdown_analysis.png'}")


# Plot drawdowns
plot_drawdowns(all_results, sizing='equal', freq='M')

### 10.3 Monthly Returns Heatmap

In [ ]:
def plot_monthly_returns_heatmap(result: BacktestResult, strategy_name: str):
    """
    Create a heatmap of monthly returns for a single strategy.
    """
    # Calculate monthly returns
    equity = result.equity_curve.resample('ME').last()
    monthly_returns = equity.pct_change().dropna()
    
    # Create pivot table (year x month)
    monthly_returns_df = pd.DataFrame({
        'year': monthly_returns.index.year,
        'month': monthly_returns.index.month,
        'return': monthly_returns.values
    })
    
    pivot = monthly_returns_df.pivot(index='year', columns='month', values='return')
    pivot.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][:len(pivot.columns)]
    
    # Plot heatmap
    fig, ax = plt.subplots(figsize=(14, 5))
    
    sns.heatmap(pivot * 100, annot=True, fmt='.1f', center=0,
                cmap='RdYlGn', ax=ax, cbar_kws={'label': 'Return (%)'})
    
    ax.set_title(f'Monthly Returns Heatmap: {strategy_name}', fontsize=14)
    ax.set_xlabel('Month')
    ax.set_ylabel('Year')
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / f'monthly_returns_{strategy_name.replace("/", "_")}.png', 
                dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved to {PLOTS_DIR}")


# Plot for best strategy
if len(results_summary) > 0:
    best_strategy = results_summary.iloc[0]['Strategy']
    if best_strategy in all_results:
        plot_monthly_returns_heatmap(all_results[best_strategy], best_strategy)

### 10.4 Ratio Distribution Histograms

In [ ]:
def plot_ratio_distributions(ratios_df: pd.DataFrame):
    """
    Plot histograms of the three financial ratios.
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    ratio_cols = ['debt_mktcap', 'roi', 'pe']
    titles = ['Debt / Market Cap', 'Return on Investment', 'Price / Earnings']
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for ax, col, title, color in zip(axes, ratio_cols, titles, colors):
        # Winsorize for visualization
        data = ratios_df[col].dropna()
        lower, upper = data.quantile(0.01), data.quantile(0.99)
        data_clipped = data[(data >= lower) & (data <= upper)]
        
        ax.hist(data_clipped, bins=50, color=color, alpha=0.7, edgecolor='white')
        ax.axvline(data_clipped.median(), color='red', linestyle='--', 
                   label=f'Median: {data_clipped.median():.2f}')
        ax.axvline(data_clipped.mean(), color='black', linestyle='-', 
                   label=f'Mean: {data_clipped.mean():.2f}')
        ax.set_title(title, fontsize=12)
        ax.set_xlabel('Value')
        ax.set_ylabel('Frequency')
        ax.legend(fontsize=8)
    
    plt.suptitle('Financial Ratio Distributions (Winsorized 1%-99%)', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'ratio_distributions.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved to {PLOTS_DIR / 'ratio_distributions.png'}")


# Plot ratio distributions
plot_ratio_distributions(ratios_df)

### 10.5 Performance Comparison Bar Chart

In [ ]:
def plot_performance_comparison(results_summary: pd.DataFrame, top_n: int = 10):
    """
    Create bar chart comparing top strategies.
    """
    top_results = results_summary.head(top_n).copy()
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Sharpe Ratio
    ax = axes[0, 0]
    bars = ax.barh(range(len(top_results)), top_results['Sharpe Ratio'], color='steelblue')
    ax.set_yticks(range(len(top_results)))
    ax.set_yticklabels(top_results['Strategy'], fontsize=8)
    ax.set_xlabel('Sharpe Ratio')
    ax.set_title('Sharpe Ratio (Top 10)')
    ax.invert_yaxis()
    
    # Total Return
    ax = axes[0, 1]
    colors = ['green' if r > 0 else 'red' for r in top_results['Total Return']]
    ax.barh(range(len(top_results)), top_results['Total Return'] * 100, color=colors)
    ax.set_yticks(range(len(top_results)))
    ax.set_yticklabels(top_results['Strategy'], fontsize=8)
    ax.set_xlabel('Total Return (%)')
    ax.set_title('Total Return (Top 10)')
    ax.invert_yaxis()
    
    # Max Drawdown
    ax = axes[1, 0]
    ax.barh(range(len(top_results)), -top_results['Max Drawdown'] * 100, color='crimson')
    ax.set_yticks(range(len(top_results)))
    ax.set_yticklabels(top_results['Strategy'], fontsize=8)
    ax.set_xlabel('Max Drawdown (%)')
    ax.set_title('Max Drawdown (Top 10)')
    ax.invert_yaxis()
    
    # Win Rate
    ax = axes[1, 1]
    ax.barh(range(len(top_results)), top_results['Win Rate'] * 100, color='goldenrod')
    ax.set_yticks(range(len(top_results)))
    ax.set_yticklabels(top_results['Strategy'], fontsize=8)
    ax.set_xlabel('Win Rate (%)')
    ax.set_title('Win Rate (Top 10)')
    ax.axvline(x=50, color='gray', linestyle='--', alpha=0.5)
    ax.invert_yaxis()
    
    plt.suptitle('Strategy Performance Comparison', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'performance_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved to {PLOTS_DIR / 'performance_comparison.png'}")


# Plot performance comparison
plot_performance_comparison(results_summary)

---

## 11. Conclusions and Interpretation

### 11.1 Key Findings

In [ ]:
# Generate automated key findings
print("=" * 80)
print("KEY FINDINGS")
print("=" * 80)

if len(results_summary) > 0:
    # Best overall strategy
    best = results_summary.iloc[0]
    print(f"\n### Best Performing Strategy ###")
    print(f"Name: {best['Strategy']}")
    print(f"Sharpe Ratio: {best['Sharpe Ratio']:.3f}")
    print(f"Total Return: {best['Total Return']:.2%}")
    print(f"Max Drawdown: {best['Max Drawdown']:.2%}")
    print(f"Win Rate: {best['Win Rate']:.2%}")
    
    # Best signal
    signal_avg = results_summary.groupby('Signal')['Sharpe Ratio'].mean()
    best_signal = signal_avg.idxmax()
    print(f"\n### Best Signal (by avg Sharpe) ###")
    print(f"Signal: {best_signal}")
    print(f"Average Sharpe: {signal_avg[best_signal]:.3f}")
    
    # Best sizing
    sizing_avg = results_summary.groupby('Sizing')['Sharpe Ratio'].mean()
    best_sizing = sizing_avg.idxmax()
    print(f"\n### Best Sizing Method (by avg Sharpe) ###")
    print(f"Method: {best_sizing}")
    print(f"Average Sharpe: {sizing_avg[best_sizing]:.3f}")
    
    # Best frequency
    freq_avg = results_summary.groupby('Frequency')['Sharpe Ratio'].mean()
    best_freq = freq_avg.idxmax()
    print(f"\n### Best Rebalancing Frequency ###")
    print(f"Frequency: {'Monthly' if best_freq == 'M' else 'Weekly'}")
    print(f"Average Sharpe: {freq_avg[best_freq]:.3f}")
else:
    print("No results available for analysis.")

### 11.2 Subjective Decisions and Justifications

This section documents key design choices made during implementation:

| Decision | Choice | Justification |
|----------|--------|---------------|
| **Rebalance Dates** | Month-end (M), Friday (W) | Month-end aligns with financial reporting cycles; Friday captures weekly price drift |
| **Quantile Thresholds** | 10th/90th percentile (decile) | Standard academic approach; balances diversification with signal strength |
| **Lookback for Δratio** | 21 trading days (~1 month) | Matches monthly rebalance frequency; captures meaningful fundamental changes |
| **Funding Rate** | 2% constant | Reasonable approximation of historical short-term rates over 2018-2023 |
| **Repo Spread** | 100bp below funding | Standard institutional rebate for shorts |
| **Winsorization** | 1%-99% | Removes extreme outliers while preserving signal distribution |
| **Minimum Universe Size** | 20 stocks on rebalance date | Ensures meaningful decile construction |
| **Signal Inversion** | Debt/MktCap ↓, P/E ↓, ROI ↑ | Follows traditional value/quality factor interpretation |
| **Combined Score** | Equal-weighted z-scores | Simple, interpretable; no look-ahead optimization of weights |

### 11.3 Interpretation of Results

In [ ]:
print("""
===================================================================================
INTERPRETATION OF RESULTS
===================================================================================

1. SIGNAL EFFICACY
------------------
The financial ratios provide varying degrees of predictive power for future returns:

- **Debt/Market Cap**: Measures leverage. Low-debt companies may be safer in downturns,
  but high-debt firms can benefit from leverage during bull markets.

- **ROI (Return on Investment)**: Measures operating efficiency. Higher ROI suggests 
  better capital allocation, though extremely high ROI may indicate unsustainable levels.

- **P/E (Price/Earnings)**: Classic valuation metric. Low P/E stocks are "value" plays;
  however, P/E can be distorted by cyclical earnings or one-time items.

- **Combined Score**: Diversifies across all three factors. May smooth out noise but
  dilutes the strongest individual signal.

2. LEVELS vs CHANGES
--------------------
Using ratio CHANGES (Δ) instead of absolute LEVELS tests whether improving fundamentals
predict returns better than current fundamental position. Changes capture momentum in
company quality improvement.

3. POSITION SIZING IMPACT
-------------------------
- **Vigintile Doubling**: Betting more on extreme values works if the signal is strong
  and extreme values are not errors.
  
- **Vigintile Halving**: Reduces exposure to outliers, works if extreme values are
  often due to data errors or temporary distortions.

4. REBALANCING FREQUENCY
------------------------
- **Monthly**: Lower transaction costs, slower adaptation to new information
- **Weekly**: Higher turnover costs, faster reaction to price/fundamental changes

The optimal frequency depends on signal decay rate and transaction cost assumptions.

===================================================================================
""")

### 11.4 Limitations and Extensions

In [ ]:
print("""
===================================================================================
LIMITATIONS
===================================================================================

1. **Transaction Costs**: Not explicitly modeled. Real strategies incur bid-ask
   spreads, market impact, and commissions that reduce returns.

2. **Survivorship Bias**: Universe is constructed from currently available tickers,
   which may exclude delisted/bankrupt companies (though Zacks data mitigates this).

3. **Look-Ahead in Universe Selection**: Universe filters use the full date range,
   which could introduce subtle look-ahead bias.

4. **Filing Date Approximation**: Some fundamental data may have been available
   slightly earlier or later than the official filing date.

5. **Capacity Constraints**: Small-cap stocks in the universe may not support
   institutional-scale positions without significant market impact.

6. **Single Period**: Results are specific to 2018-2023, which includes COVID crash/
   recovery. Performance in other regimes may differ substantially.

===================================================================================
POTENTIAL EXTENSIONS
===================================================================================

1. **Sector-Neutral Implementation**: Rank stocks within sectors to neutralize
   sector exposure.

2. **Dynamic Signal Weights**: Learn optimal weights for combined score using
   rolling window regression.

3. **Machine Learning Integration**: Use gradient boosting or neural networks
   to combine signals non-linearly.

4. **Transaction Cost Modeling**: Add realistic trading costs to find optimal
   rebalance frequency.

5. **Risk Parity Position Sizing**: Size positions inversely to volatility for
   more stable risk contribution.

6. **Alternative Ratios**: Add other metrics (free cash flow yield, earnings
   quality, accruals) to signal set.

===================================================================================
""")

### 11.5 Final Summary Table

In [ ]:
# Final summary table with formatting
print("=" * 100)
print("FINAL RESULTS SUMMARY - ALL STRATEGIES")
print("=" * 100)

# Display full results table
display_cols = ['Strategy', 'Sharpe Ratio', 'Total Return', 'Max Drawdown', 'Win Rate', 'Calmar Ratio']
print(results_summary[display_cols].to_string(index=False))

# Save final results
final_file = RESULTS_DIR / 'final_summary.csv'
results_summary.to_csv(final_file, index=False)
print(f"\n✓ Final results saved to {final_file}")

# Pickle all results for future analysis
import pickle
with open(RESULTS_DIR / 'all_backtest_results.pkl', 'wb') as f:
    pickle.dump(all_results, f)
print(f"✓ Full backtest objects saved to {RESULTS_DIR / 'all_backtest_results.pkl'}")

---

## Notebook Execution Complete

**Summary:**
- Loaded ~1200 ticker universe after applying 5 filters
- Computed 3 financial ratios (Debt/MktCap, ROI, P/E) + combined score
- Ran 42 strategy variants (7 signals × 2 frequencies × 3 sizing methods)
- Generated comprehensive performance metrics and visualizations

**Output Files:**
- `outputs/results/universe_tickers.csv` - Universe definition
- `outputs/results/computed_ratios.parquet` - Daily ratio values
- `outputs/results/ratio_changes.parquet` - Ratio change values
- `outputs/results/backtest_results.csv` - Summary metrics
- `outputs/results/final_summary.csv` - Final results
- `outputs/results/all_backtest_results.pkl` - Full backtest objects
- `outputs/plots/*.png` - Visualization files